# Analysis on transformed data

## Splitting data by experiment

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score

In [2]:
df = pd.read_csv('../../data/raw_post_multi_extract/transformed/objectLevel/labelingInstanceGazeInfo_jun19.csv')

# /Users/jacobrivera/Documents/GitHub/DiLabWork/data/raw_post_multi_extract/transformed/objectLevel/labelingInstanceGazeInfo_jun19.csv



In [3]:
df_185 = df[df['expID'] == 185]
df_186 = df[df['expID'] == 186]
df_187 = df[df['expID'] == 187]
df_188 = df[df['expID'] == 188]

df_185_pre = df_185[df_185['pretrained'] == 1]
df_185_not = df_185[df_185['pretrained'] == 0]

df_186_pre = df_186[df_186['pretrained'] == 1]
df_186_not = df_186[df_186['pretrained'] == 0]

df_187_pre = df_187[df_187['pretrained'] == 1]
df_187_not = df_187[df_187['pretrained'] == 0]

df_188_pre = df_188[df_188['pretrained'] == 1]
df_188_not = df_188[df_188['pretrained'] == 0]

proportion_cols = ['i1_curr_targetProp', 'i2_curr_targetProp', 'i3_curr_targetProp',
                    'i4_curr_targetProp', 'i5_curr_targetProp', 'i6_curr_targetProp']

all_dfs = [df_185_pre, df_185_not, 
            df_186_pre, df_186_not,
            df_187_pre, df_187_not, 
            df_188_pre, df_188_not,
            df_185, df_186, df_187, df_188]


In [4]:
'''Index(['subID', 'expID', 'object', 'learned', 'pretrained',
       'i1_curr_targetObj', 'i1_curr_targetProp', 'i1_curr_distObj_1',
       'i1_curr_distProp_1', 'i1_curr_distObj_2', 'i1_curr_distProp_2',
       'i1_curr_distObj_3', 'i1_curr_distProp_3', 'i1_prev_target',
       'i1_prev_dist_1', 'i1_prev_dist_2', 'i1_prev_dist_3',
       'i2_curr_targetObj', 'i2_curr_targetProp', 'i2_curr_distObj_1',
       'i2_curr_distProp_1', 'i2_curr_distObj_2', 'i2_curr_distProp_2',
       'i2_curr_distObj_3', 'i2_curr_distProp_3', 'i2_prev_target',
       'i2_prev_dist_1', 'i2_prev_dist_2', 'i2_prev_dist_3',
       'i3_curr_targetObj', 'i3_curr_targetProp', 'i3_curr_distObj_1',
       'i3_curr_distProp_1', 'i3_curr_distObj_2', 'i3_curr_distProp_2',
       'i3_curr_distObj_3', 'i3_curr_distProp_3', 'i3_prev_target',
       'i3_prev_dist_1', 'i3_prev_dist_2', 'i3_prev_dist_3',
       'i4_curr_targetObj', 'i4_curr_targetProp', 'i4_curr_distObj_1',
       'i4_curr_distProp_1', 'i4_curr_distObj_2', 'i4_curr_distProp_2',
       'i4_curr_distObj_3', 'i4_curr_distProp_3', 'i4_prev_target',
       'i4_prev_dist_1', 'i4_prev_dist_2', 'i4_prev_dist_3',
       'i5_curr_targetObj', 'i5_curr_targetProp', 'i5_curr_distObj_1',
       'i5_curr_distProp_1', 'i5_curr_distObj_2', 'i5_curr_distProp_2',
       'i5_curr_distObj_3', 'i5_curr_distProp_3', 'i5_prev_target',
       'i5_prev_dist_1', 'i5_prev_dist_2', 'i5_prev_dist_3',
       'i6_curr_targetObj', 'i6_curr_targetProp', 'i6_curr_distObj_1',
       'i6_curr_distProp_1', 'i6_curr_distObj_2', 'i6_curr_distProp_2',
       'i6_curr_distObj_3', 'i6_curr_distProp_3', 'i6_prev_target',
       'i6_prev_dist_1', 'i6_prev_dist_2', 'i6_prev_dist_3'],'''

"Index(['subID', 'expID', 'object', 'learned', 'pretrained',\n       'i1_curr_targetObj', 'i1_curr_targetProp', 'i1_curr_distObj_1',\n       'i1_curr_distProp_1', 'i1_curr_distObj_2', 'i1_curr_distProp_2',\n       'i1_curr_distObj_3', 'i1_curr_distProp_3', 'i1_prev_target',\n       'i1_prev_dist_1', 'i1_prev_dist_2', 'i1_prev_dist_3',\n       'i2_curr_targetObj', 'i2_curr_targetProp', 'i2_curr_distObj_1',\n       'i2_curr_distProp_1', 'i2_curr_distObj_2', 'i2_curr_distProp_2',\n       'i2_curr_distObj_3', 'i2_curr_distProp_3', 'i2_prev_target',\n       'i2_prev_dist_1', 'i2_prev_dist_2', 'i2_prev_dist_3',\n       'i3_curr_targetObj', 'i3_curr_targetProp', 'i3_curr_distObj_1',\n       'i3_curr_distProp_1', 'i3_curr_distObj_2', 'i3_curr_distProp_2',\n       'i3_curr_distObj_3', 'i3_curr_distProp_3', 'i3_prev_target',\n       'i3_prev_dist_1', 'i3_prev_dist_2', 'i3_prev_dist_3',\n       'i4_curr_targetObj', 'i4_curr_targetProp', 'i4_curr_distObj_1',\n       'i4_curr_distProp_1', 'i4_curr_

In [4]:
def plot_object_subject_target(df, object, experiment, subject, learner_type):
    object_groups = df.groupby('object')
    target_columns = ['subID', 'expID', 'object', 'learned', 'pretrained',
                  'i1_curr_targetProp', 'i2_curr_targetProp', 'i3_curr_targetProp',
                  'i4_curr_targetProp', 'i5_curr_targetProp', 'i6_curr_targetProp']
    target_df_obj = object_groups.get_group(object)[target_columns]

    target_df_obj_exp = target_df_obj[(target_df_obj['expID'] == experiment) & (target_df_obj['subID'] == subject)]

    proportion_cols = ['i1_curr_targetProp', 'i2_curr_targetProp', 'i3_curr_targetProp',
                  'i4_curr_targetProp', 'i5_curr_targetProp', 'i6_curr_targetProp']

    domain = np.array([1, 2, 3, 4, 5, 6]).reshape(-1,1)
    plt.scatter(domain, target_df_obj_exp[proportion_cols], label=subject)

    X = np.array(target_df_obj_exp[proportion_cols].values.flatten().tolist()).reshape(-1, 1)

    reg = LinearRegression().fit(domain, X)

    y_pred = [reg.predict([[i]]) for i in range(1,7)]
    flatten_y_pred = [j for sub in y_pred for j in sub]

    plt.plot(domain, flatten_y_pred, label='regression')
    plt.xlabel('instance')
    plt.ylabel('gaze proportion')
    plt.title('{level} learners: target obj {obj} gaze prop per labeling instances'.format(level=learner_type,obj=object))
    plt.legend()

In [5]:

def grapher(df, obj):
    proportion_cols = ['i1_curr_targetProp', 'i2_curr_targetProp', 'i3_curr_targetProp',
                    'i4_curr_targetProp', 'i5_curr_targetProp', 'i6_curr_targetProp']
    domain = np.array([1, 2, 3, 4, 5, 6]).reshape(-1,1)

    for sub in df['subID'].unique():
        print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
        plt.scatter(domain, df[(df['object'] == obj) & (df['subID'] == sub)][proportion_cols])
        plt.show()

        if (df[(df['object'] == obj) & (df['subID'] == sub)]['learned'].values == 1):
            print("LEARNED")
        else:
            print("NOT LEARNED")
        print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

# grapher(df_188_not, 1)


In [5]:
prop_cols = [
        'i1_curr_targetProp', 'i1_curr_distProp_1',  'i1_curr_distProp_2' ,'i1_curr_distProp_3',
        'i1_prev_target', 'i1_prev_dist_1', 'i1_prev_dist_2', 'i1_prev_dist_3',
        'i2_curr_targetProp', 'i2_curr_distProp_1', 'i2_curr_distProp_2', 'i2_curr_distProp_3',
        'i2_prev_target', 'i2_prev_dist_1', 'i2_prev_dist_2', 'i2_prev_dist_3',
        'i3_curr_targetProp', 'i3_curr_distProp_1',  'i3_curr_distProp_2', 'i3_curr_distProp_3',
        'i3_prev_target', 'i3_prev_dist_1', 'i3_prev_dist_2', 'i3_prev_dist_3',
        'i4_curr_targetProp', 'i4_curr_distProp_1',  'i4_curr_distProp_2',
        'i4_curr_distProp_3', 'i4_prev_target', 'i4_prev_dist_1', 'i4_prev_dist_2', 'i4_prev_dist_3',
        'i5_curr_targetProp', 'i5_curr_distProp_1', 'i5_curr_distProp_2','i5_curr_distProp_3',
        'i5_prev_target', 'i5_prev_dist_1', 'i5_prev_dist_2', 'i5_prev_dist_3',
        'i6_curr_targetProp','i6_curr_distProp_1',  'i6_curr_distProp_2', 'i6_curr_distProp_3',
        'i6_prev_target','i6_prev_dist_1', 'i6_prev_dist_2', 'i6_prev_dist_3']

distractor_cols  = [
        'i1_curr_targetProp', 'i1_curr_distProp_1', 'i1_curr_distProp_2', 'i1_curr_distProp_3',
        'i2_curr_targetProp', 'i2_curr_distProp_1', 'i2_curr_distProp_2', 'i2_curr_distProp_3',
        'i3_curr_targetProp', 'i3_curr_distProp_1', 'i3_curr_distProp_2', 'i3_curr_distProp_3',
        'i4_curr_targetProp', 'i4_curr_distProp_1', 'i4_curr_distProp_2', 'i4_curr_distProp_3',
        'i5_curr_targetProp', 'i5_curr_distProp_1', 'i5_curr_distProp_2', 'i5_curr_distProp_3',
        'i6_curr_targetProp', 'i6_curr_distProp_1', 'i6_curr_distProp_2', 'i6_curr_distProp_3',]

distractor_cols_w_pre  = [ 'pretrained',
        'i1_curr_targetProp', 'i1_curr_distProp_1', 'i1_curr_distProp_2', 'i1_curr_distProp_3',
        'i2_curr_targetProp', 'i2_curr_distProp_1', 'i2_curr_distProp_2', 'i2_curr_distProp_3',
        'i3_curr_targetProp', 'i3_curr_distProp_1', 'i3_curr_distProp_2', 'i3_curr_distProp_3',
        'i4_curr_targetProp', 'i4_curr_distProp_1', 'i4_curr_distProp_2', 'i4_curr_distProp_3',
        'i5_curr_targetProp', 'i5_curr_distProp_1', 'i5_curr_distProp_2', 'i5_curr_distProp_3',
        'i6_curr_targetProp', 'i6_curr_distProp_1', 'i6_curr_distProp_2', 'i6_curr_distProp_3',]

curr_prop_cols = ['i1_curr_targetProp', 'i2_curr_targetProp', 'i3_curr_targetProp',
                    'i4_curr_targetProp', 'i5_curr_targetProp', 'i6_curr_targetProp']

curr_prop_cols_w_pre = ['pretrained',
                        'i1_curr_targetProp', 'i2_curr_targetProp', 'i3_curr_targetProp',
                        'i4_curr_targetProp', 'i5_curr_targetProp', 'i6_curr_targetProp']

all_dfs = [df_185_pre, df_185_not, 
            df_186_pre, df_186_not,
            df_187_pre, df_187_not, 
            df_188_pre, df_188_not,
            df_185, df_186, df_187, df_188]
all_dfs_strings = ['df_185_pre', 'df_185_not', 
            'df_186_pre', 'df_186_not',
            'df_187_pre', 'df_187_not', 
            'df_188_pre', 'df_188_not',
            'df_185', 'df_186', 'df_187', 'df_188']

col_array = [curr_prop_cols,curr_prop_cols_w_pre, distractor_cols, distractor_cols_w_pre]


In [6]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score
data = all_dfs[1] # df_185_not

col_array_names = ["proportions of current target", "proportions of current target including pretrained col", 
                   "proportions of current target and distractors", "proportions of current target and distractors including pretrained col"]
# kernels = ['linear','rbf','sigmoid','poly']
kernels = ['rbf']


for i in range(len(all_dfs)):
    for j in range(len(col_array)):
        x = all_dfs[i][col_array[j]]
        y = all_dfs[i]['pretrained']

        if len(x) == 0: continue
        if len(y.unique()) == 1: continue
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

        for kern in kernels:
            clf = svm.SVC(kernel=kern)
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            print()
            print('Data subset: {sub}\nColumns used are the {cols}\nKernel: {kernel}'.format(sub=all_dfs_strings[i],cols=col_array_names[j], kernel=kern))
            print(classification_report(y_pred=y_pred, y_true=y_test))
            # print( "Accuracy: ", balanced_accuracy_score(y_test, y_pred))
            print()




Data subset: df_186
Columns used are the proportions of current target
Kernel: rbf
              precision    recall  f1-score   support

           0       0.85      1.00      0.92       187
           1       0.00      0.00      0.00        33

    accuracy                           0.85       220
   macro avg       0.42      0.50      0.46       220
weighted avg       0.72      0.85      0.78       220



Data subset: df_186
Columns used are the proportions of current target including pretrained col
Kernel: rbf
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       187
           1       1.00      1.00      1.00        33

    accuracy                           1.00       220
   macro avg       1.00      1.00      1.00       220
weighted avg       1.00      1.00      1.00       220



Data subset: df_186
Columns used are the proportions of current target and distractors
Kernel: rbf
              precision    recall  f1-score   suppo

c:\Users\jacob\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\jacob\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\jacob\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [7]:

df_train = df_185

X = df_train[distractor_cols]
y = df_train['learned']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = svm.SVC(kernel="rbf")
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(classification_report(y_pred=y_pred, y_true=y_test))

              precision    recall  f1-score   support

           0       0.82      0.97      0.89       173
           1       0.69      0.23      0.35        47

    accuracy                           0.81       220
   macro avg       0.76      0.60      0.62       220
weighted avg       0.79      0.81      0.78       220



In [25]:
from sklearn import tree
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import numpy as np


x_str = 'Experiment 186, not trained rows, distractor columns included in training\n\t pretrained column NOT included in training'
x = df_187[curr_prop_cols_w_pre]
y = df_187['learned']


X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

clf = LogisticRegression()
clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)


print('Dataset: {data}'.format(data=x_str))
print(classification_report(y_pred=y_pred, y_true=y_test))



Dataset: Experiment 186, not trained rows, distractor columns included in training
	 pretrained column NOT included in training
              precision    recall  f1-score   support

           0       0.66      0.35      0.46        54
           1       0.82      0.94      0.87       166

    accuracy                           0.80       220
   macro avg       0.74      0.65      0.67       220
weighted avg       0.78      0.80      0.77       220



In [28]:
X_train, X_test, y_train, y_test = train_test_split(df_188[curr_prop_cols_w_pre], df_188['learned'], test_size=0.2, random_state=42)
y_pred = clf.predict(X_test)
print(classification_report(y_pred=y_pred, y_true=y_test))

              precision    recall  f1-score   support

           0       0.42      0.17      0.24        30
           1       0.88      0.96      0.92       186

    accuracy                           0.85       216
   macro avg       0.65      0.56      0.58       216
weighted avg       0.81      0.85      0.82       216



In [35]:
from sklearn.ensemble import AdaBoostClassifier

x = df[curr_prop_cols_w_pre]
y = df['learned']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

clfa = AdaBoostClassifier(n_estimators=100, random_state=0)
clfa.fit(X_train, y_train)
y_pred = clfa.predict(X_test)
print(classification_report(y_pred=y_pred, y_true=y_test))

              precision    recall  f1-score   support

           0       0.70      0.72      0.71       334
           1       0.82      0.81      0.82       541

    accuracy                           0.78       875
   macro avg       0.76      0.76      0.76       875
weighted avg       0.78      0.78      0.78       875



In [40]:
y_pred = clfa.predict(df_188[curr_prop_cols_w_pre])
print(classification_report(y_pred=y_pred, y_true=df_188['learned']))

              precision    recall  f1-score   support

           0       0.48      0.56      0.52       145
           1       0.93      0.91      0.92       935

    accuracy                           0.86      1080
   macro avg       0.71      0.73      0.72      1080
weighted avg       0.87      0.86      0.86      1080



In [42]:
clfa.feature_importances_

array([0.03, 0.1 , 0.22, 0.13, 0.25, 0.16, 0.11])